# Getting and Cleaning Oklahoma Production Data

<!--PELICAN_BEGIN_SUMMARY-->

follow the principles of tidy data in Jeff Leek's [guide to data sharing](https://github.com/jtleek/datasharing).

Hadley Wickham's principles of [Tidy Data](http://vita.had.co.nz/papers/tidy-data.pdf)

In part 1, we're going to obtain the raw data from OCC.  Examine the contents of the archive, look at the different files types.

Combine the raw data into a single combined file that we'll use to create tidy datasets of production information (and well location, maybe).

<!-- PELICAN_END_SUMMARY -->

## The Raw Data

### another title

downloading the file from the [Oklahoma Corporation Commission](http://www.occeweb.com/og/ogdatafiles2.htm) website.  We will download the `Historical.zip` file, which is stated to contain:

```Oil/Gas Production Historical Records includes All Production Available from 1987 to 2012 Vertical Bar Delimited```.

File is almost 80 MB.

Extract the zip archive to `data/historical/`.

In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 10

data_dir = 'data/historical/'

In [ ]:
import urllib
urllib.urlretrieve ('ftp://ftp.occ.state.ok.us/OG_DATA/historical.ZIP', 'data/historical.zip')

# For Python 3, try:
# import urllib.request 
# urllib.request.urlretrieve(url, filename)

import zipfile
zip_ref = zipfile.ZipFile('data/historical.zip', 'r')
zip_ref.extractall(data_dir)
zip_ref.close()

Take a look at dir contents

In [2]:
import os
os.listdir('data/historical/')

['1987prodn.txt',
 '1988prodn.txt',
 '1989prodn.txt',
 '1990prodn.txt',
 '1991prodn.txt',
 '1992prodn.txt',
 '1993prodn.txt',
 '1994PRODN.TXT',
 '1995prodn.txt',
 '1996prodn.txt',
 '1997prodn.txt',
 '1998prodn.txt',
 '1999prodn.txt',
 '2000prodn.txt',
 '2001prodn.txt',
 '2002prodn.txt',
 '2003PRODN.TXT',
 '2004PRODN.TXT',
 '2005PRODN.TXT',
 '2006prodn.txt',
 '2007PRODN.TXT',
 '2008PRODN.TXT',
 '2009PRODN.txt',
 '2010 rbdms extract2013.txt',
 '2010PRODN.txt',
 '2011 rbdms extract2013.txt',
 '2011PRODN.txt',
 '2012PRODN.txt',
 '2013prodn.txt',
 '2014prodn.txt',
 '2015prodn.txt']

Production from 1987 to 2015.  We have re-extracted data from 2010 and 2011.  Why?

Look at contents of early files.

In [3]:
def show_file(filename, nrows):
    from itertools import islice
    with open(filename) as f:
        for line in islice(f, nrows):
            print line

show_file("data/historical/1987prodn.txt", 5)



API_COUNTY|API_NUMBER|SUFFIX|WELL_NAME                     |WELL_NO|  OPER_NO|OPER_SUFFIX|OPERATOR                                |ME|  SECTION|TWP|RAN|Q4|Q3|Q2|Q1| LATITUDE|LONGITUDE|OTC_COUNTY|OTC_LEASE_NO|OTC_SUB_NO|OTC_MERGE|  POOL_NO|CODE    |FORMATION                               |OFB|ALLOWABLE_CLASS|ALLOWABLE_TYPE| PURCH_NO|PURCHASER                               |PURCH_SUFFIX|OFB|     YEAR|      JAN|      OIL|      GAS|      FEB|      OIL|      GAS|      MAR|      OIL|      GAS|      APR|      OIL|      GAS|      MAY|      OIL|      GAS|      JUN|      OIL|      GAS|      JUL|      OIL|      GAS|      AUG|      OIL|      GAS|      SEP|      OIL|      GAS|      OCT|      OIL|      GAS|      NOV|      OIL|      GAS|      DEC|      OIL|      GAS     

----------|----------|------|------------------------------|-------|---------|-----------|----------------------------------------|--|---------|---|---|--|--|--|--|---------|---------|----------|------------|----------|---------|-

Explain this file format. 69 column types, storing all different kinds of data.  Many not used.  Rich data set, but could certainly use a bit of tidying.

Look at more recent files:

In [4]:
show_file("data/historical/2015prodn.txt", 5)

API_COUNTY|API_NUMBER|S|WELL_NAME|WELL_NO|OPER_NO|OPERATOR|ME|SECTION|TWP|RAN|Q4|Q3|Q2|Q1|LATITUDE|LONGITUDE|OTC_COUNTY|OTC_LEASE_NO|OTC_SUB_NO|OTC_MERGE|POOL_NO|CODE|FORMATION|OFB|ALLOWABLE_CLASS|ALLOWABLE_TYPE|PURCH_NO|PURCHASER|YEAR|JAN|OIL|GAS|FEB|OIL|GAS|MAR|OIL|GAS|APR|OIL|GAS|MAY|OIL|GAS|JUN|OIL|GAS|JUL|OIL|GAS|AUG|OIL|GAS|SEP|OIL|GAS|OCT|OIL|GAS|NOV|OIL|GAS|DEC|OIL|GAS

003|00001||KIRCHER                       |     1|4030|CHAMPLIN EXPLORATION INC|INDIAN|4|27N|11W|||CNE4|NW4|36.8532|-98.3895|003|||||351MISS|MISSISSIPPI||||||2015|1||0|2||12|3||0|4||0|5|||6|||7|||8|||9|||10|||11|||12||

003|00025||HAGUE                         |     1|19694|BVD INC|Indian|22|27N|11W|    |    |CSE4|SE4 |36.7989|-98.3624|003|||||404CHRK|CHEROKEE||||||2015|1||81|2||69|3||60|4||38|5||42|6||45|7||36|8||32|9||31|10||39|11||37|12||45

003|00068||ADAMS   "A"                   |1-3   |17441|CHESAPEAKE OPERATING LLC|Indian|3|25N|10W|    |    |SW4 |SW4 |36.6683|-98.2678|003|||||404OSWGU|OSWEGO UP||||||2015|

Discuss this file format.

American Petroleum Institute defines an API well number.  Details on this can be found on [wikipedia](https://en.wikipedia.org/wiki/API_well_number), DrillingInfo also has a nice [summary](https://info.drillinginfo.com/api-number-dead-long-live-us-well-number/). 
First ten digits give the well's surface locatin.

First two digits are the State Code. 35 for OK.

Next three digits are the County Code.  API_COUNTY field.

Next five digits are the Unique Well Identifier.  API_Number field.

Next two digits (digits 11 and 12) are the Directional Sidetrack Codes.
No official standard for final two, other than being reserved for Event Sequence Codes.

In [8]:
#generate a list of years that will be used to form filenames
years = np.arange(1988, 2016, 1)

#list to track the dataframes from each year that will be 
#concatenated after processing
annual_data = {}

for year in years:
    
    #data for 1994 is missing, so ignore this year
    if (year == 1994):
        continue
    
    filename = data_dir + str(year)+'prodn.txt'
    print('Reading %s'%filename)
    
    #the file format is different for years 2008 and before
    #so these files need to be handled differently
    #
    #there are 6 lines in years between 1990 and 1995 that have
    #an extra field.  This could be due to an extra delimiter
    #'|' in the line.  These are ignored.
    if year <= 2008:
        data = pd.read_csv(filename, sep="|", engine='c', 
                        doublequote=False,error_bad_lines=False,
                        low_memory=False)
        
        #remove the first row with ------
        data = data.ix[1:]
        
        #remove the whitespace from the column names
        data = data.rename(columns=lambda x: x.strip())
        
        #these columns don't appear in the later years, so drop
        #them now for consistency
        data.drop(['PURCH_SUFFIX', 'OPER_SUFFIX', 'OFB.1'], axis=1, 
                inplace=True)
                
        #the last Gas column for December will be incorrectly
        #named because of whitespace.  Rename it manually now
        data.columns.values[-1] = 'GAS.11'
    else:
        #the files for 2009 and later are much cleaner and 
        #require less processing
        
        #Sometimes there is an extra line at the top at the file
        # that contains the string 'ExtractRecord'.  If this 
        # is the case, ignore the first line
        with open(filename, 'r') as f:
            first_line = f.readline()
        if 'API_NUMBER' not in first_line:
            data = pd.read_csv(filename, sep="|", engine='c', 
            doublequote=False,error_bad_lines=False, skiprows=1)
        else:
            data = pd.read_csv(filename, sep="|", engine='c', 
                        doublequote=False,error_bad_lines=False)

    # remove any entries that don't have a valid well number
    # some files (like 2004 & 2005) have a row with 
    # '39949 rows selected.' at the bottom.  Most of this row
    # will have NaN values, and will cause problems.  Remove
    # them.
    data = data[pd.notnull(data['API_NUMBER'])]
    
    annual_data[year] = data

Reading data/historical/1988prodn.txt
Reading data/historical/1989prodn.txt
Reading data/historical/1990prodn.txt


Skipping line 19661: expected 69 fields, saw 70
Skipping line 19662: expected 69 fields, saw 70



Reading data/historical/1991prodn.txt


Skipping line 20832: expected 69 fields, saw 70



Reading data/historical/1992prodn.txt


...

Reading data/historical/1993prodn.txt


Skipping line 11718: expected 69 fields, saw 70



Reading data/historical/1995prodn.txt


Skipping line 23792: expected 69 fields, saw 70



Reading data/historical/1996prodn.txt
Reading data/historical/1997prodn.txt
Reading data/historical/1998prodn.txt
Reading data/historical/1999prodn.txt
Reading data/historical/2000prodn.txt
Reading data/historical/2001prodn.txt
Reading data/historical/2002prodn.txt
Reading data/historical/2003prodn.txt
Reading data/historical/2004prodn.txt
Reading data/historical/2005prodn.txt
Reading data/historical/2006prodn.txt
Reading data/historical/2007prodn.txt
Reading data/historical/2008prodn.txt
Reading data/historical/2009prodn.txt
Reading data/historical/2010prodn.txt
Reading data/historical/2011prodn.txt
Reading data/historical/2012prodn.txt
Reading data/historical/2013prodn.txt
Reading data/historical/2014prodn.txt
Reading data/historical/2015prodn.txt


In [37]:
data = pd.concat(annual_data)

In [33]:
#Form the full unique API number by concatenating the 
# Okalhoma state code '35' with the API_COUNTY and 
# API_NUMBER fields
data['API_NUMBER'] = data['API_NUMBER'].astype(int, raise_on_error=True).apply(lambda x: '{0:0>5}'.format(x))
data['API_COUNTY'] = data['API_COUNTY'].astype(int, raise_on_error=True).apply(lambda x: '{0:0>3}'.format(x))
data['ENTITY_ID'] = '35'+data['API_COUNTY']+data['API_NUMBER']+'0000'

prod_data = data[['GAS', 'OIL', 'GAS.1', 'OIL.1', 'GAS.2', 
             'OIL.2', 'GAS.3', 'OIL.3', 'GAS.4', 'OIL.4',
             'GAS.5', 'OIL.5', 'GAS.6', 'OIL.6', 'GAS.7', 
             'OIL.7', 'GAS.8', 'OIL.8', 'GAS.9', 'OIL.9',
             'GAS.10', 'OIL.10', 'GAS.11', 'OIL.11',
             'ENTITY_ID', 'YEAR']]

#data.apply(pd.to_numeric, errors='ignore')
#convert the data type of all OIL and GAS columns to numeric
for col_name in list(prod_data.columns.values):
    if ('GAS' in col_name) or ('OIL' in col_name):
        prod_data[col_name] = pd.to_numeric(prod_data[col_name], errors='coerce')

#rename the oil and gas montly totals to something more
#descriptive
prod_data.rename(columns={'GAS': 'GAS - January',
                     'OIL': 'OIL - January',
                     'GAS.1': 'GAS - February',
                     'OIL.1': 'OIL - February',
                     'GAS.2': 'GAS - March',
                     'OIL.2': 'OIL - March',
                     'GAS.3': 'GAS - April ',
                     'OIL.3': 'OIL - April',
                     'GAS.4': 'GAS - May',
                     'OIL.4': 'OIL - May',
                     'GAS.5': 'GAS - June',
                     'OIL.5': 'OIL - June',
                     'GAS.6': 'GAS - July',
                     'OIL.6': 'OIL - July',
                     'GAS.7': 'GAS - August',
                     'OIL.7': 'OIL - August',
                     'GAS.8': 'GAS - September',
                     'OIL.8': 'OIL - September',
                     'GAS.9': 'GAS - October',
                     'OIL.9': 'OIL - October',
                     'GAS.10': 'GAS - November',
                     'OIL.10': 'OIL - November',
                     'GAS.11': 'GAS - December',
                     'OIL.11': 'OIL - December',
                     }, inplace=True)

#some entity id's have multiple entries.  I do not know 
#why this is.  it appears like they might be the production
#attributed to different owners. try summing the production
#for now...
prod_data = prod_data.groupby(by=['ENTITY_ID','YEAR']).sum()
prod_data.reset_index(inplace=True)

/Users/bhall/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [34]:
prod_data

,ENTITY_ID,YEAR,GAS - January,OIL - January,GAS - February,OIL - February,GAS - March,OIL - March,GAS - April,OIL - April,...,GAS - August,OIL - August,GAS - September,OIL - September,GAS - October,OIL - October,GAS - November,OIL - November,GAS - December,OIL - December
0,35003000010000,2009,1312.0,NaN,957.0,NaN,1242.0,NaN,1165.0,NaN,...,1091.0,NaN,1033.0,NaN,931.0,NaN,993.0,NaN,1264.0,NaN
1,35003000010000,2010,NaN,NaN,NaN,NaN,1133.0,NaN,939.0,NaN,...,673.0,NaN,823.0,NaN,871.0,NaN,906.0,NaN,878.0,NaN
2,35003000010000,2011,873.0,NaN,809.0,NaN,936.0,NaN,946.0,NaN,...,892.0,NaN,772.0,NaN,725.0,NaN,851.0,NaN,713.0,NaN
3,35003000010000,2012,829.0,NaN,766.0,NaN,491.0,NaN,564.0,NaN,...,668.0,NaN,695.0,NaN,609.0,NaN,539.0,NaN,351.0,NaN
4,35003000010000,2013,822.0,NaN,379.0,NaN,361.0,NaN,318.0,NaN,...,626.0,NaN,486.0,NaN,808.0,NaN,644.0,NaN,493.0,NaN
5,35003000010000,2014,516.0,NaN,584.0,NaN,400.0,NaN,596.0,NaN,...,591.0,NaN,719.0,NaN,751.0,NaN,635.0,NaN,388.0,NaN
6,35003000010000,2015,0.0,NaN,12.0,NaN,0.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,35003000010000,2001,2477.0,NaN,2236.0,NaN,2484.0,NaN,2389.0,NaN,...,2378.0,NaN,2123.0,NaN,2514.0,NaN,2087.0,NaN,2303.0,NaN
8,35003000010000,2002,2450.0,NaN,2073.0,NaN,2320.0,NaN,2121.0,NaN,...,1574.0,NaN,2270.0,NaN,2264.0,NaN,2097.0,NaN,2173.0,NaN
9,35003000010000,2003,2531.0,NaN,1889.0,NaN,2186.0,NaN,2072.0,NaN,...,1997.0,NaN,2094.0,NaN,2237.0,NaN,2118.0,NaN,2316.0,NaN


In [1]:
#now clean the formation string.  These tend to have 
#extraneous whitespace
def clean_text_data(in_string):

    out_string = str(in_string).strip()
    return " ".join(out_string.split())

#data["FORMATION"] = data["FORMATION"].astype(str) 
data["FORMATION"] = data["FORMATION"].apply(clean_text_data)
data["WELL_NAME"] = data["WELL_NAME"].apply(clean_text_data)
data["OPERATOR"] = data["OPERATOR"].apply(clean_text_data)
data["PURCHASER"] = data["PURCHASER"].apply(clean_text_data)
data

NameError: name 'data' is not defined

In [44]:
data.columns

Index([u'ALLOWABLE_CLASS', u'ALLOWABLE_TYPE', u'API_COUNTY', u'API_NUMBER',
       u'APR', u'AUG', u'CODE', u'DEC', u'ENTITY_ID', u'FEB', u'FORMATION',
       u'GAS', u'GAS.1', u'GAS.10', u'GAS.11', u'GAS.2', u'GAS.3', u'GAS.4',
       u'GAS.5', u'GAS.6', u'GAS.7', u'GAS.8', u'GAS.9', u'JAN', u'JUL',
       u'JUN', u'LATITUDE', u'LONGITUDE', u'MAR', u'MAY', u'ME', u'NOV',
       u'OCT', u'OFB', u'OIL', u'OIL.1', u'OIL.10', u'OIL.11', u'OIL.2',
       u'OIL.3', u'OIL.4', u'OIL.5', u'OIL.6', u'OIL.7', u'OIL.8', u'OIL.9',
       u'OPERATOR', u'OPER_NO', u'OTC_COUNTY', u'OTC_LEASE_NO', u'OTC_MERGE',
       u'OTC_SUB_NO', u'POOL_NO', u'PURCHASER', u'PURCH_NO', u'Q1', u'Q2',
       u'Q3', u'Q4', u'RAN', u'S', u'SECTION', u'SEP', u'SUFFIX', u'TWP',
       u'WELL_NAME', u'WELL_NO', u'YEAR'],
      dtype='object')

In [45]:
columns_to_keep = ['ENTITY_ID', 'FORMATION', 'LATITUDE', 'LONGITUDE', 'OPERATOR', 'WELL_NAME', 'PURCHASER']

In [30]:
annual_production_data = {}
annual_meta_data = {}

for year in annual_data:
    print('Cleaning %d data...'%year)
    data = annual_data[year]

    
    #Form the full unique API number by concatenating the 
    # Okalhoma state code '35' with the API_COUNTY and 
    # API_NUMBER fields
    data['API_NUMBER'] = data['API_NUMBER'].astype(int, raise_on_error=True).apply(lambda x: '{0:0>5}'.format(x))
    data['API_COUNTY'] = data['API_COUNTY'].astype(int, raise_on_error=True).apply(lambda x: '{0:0>3}'.format(x))
    data['ENTITY_ID'] = '35'+data['API_COUNTY']+data['API_NUMBER']+'0000'
    
    prod_data = data[['GAS', 'OIL', 'GAS.1', 'OIL.1', 'GAS.2', 
                 'OIL.2', 'GAS.3', 'OIL.3', 'GAS.4', 'OIL.4',
                 'GAS.5', 'OIL.5', 'GAS.6', 'OIL.6', 'GAS.7', 
                 'OIL.7', 'GAS.8', 'OIL.8', 'GAS.9', 'OIL.9',
                 'GAS.10', 'OIL.10', 'GAS.11', 'OIL.11',
                 'ENTITY_ID', 'YEAR']]

    #data.apply(pd.to_numeric, errors='ignore')
    #convert the data type of all OIL and GAS columns to numeric
    for col_name in list(prod_data.columns.values):
        if ('GAS' in col_name) or ('OIL' in col_name):
            prod_data[col_name] = pd.to_numeric(prod_data[col_name], errors='coerce')
    
    #rename the oil and gas montly totals to something more
    #descriptive
    prod_data.rename(columns={'GAS': 'GAS - January',
                         'OIL': 'OIL - January',
                         'GAS.1': 'GAS - February',
                         'OIL.1': 'OIL - February',
                         'GAS.2': 'GAS - March',
                         'OIL.2': 'OIL - March',
                         'GAS.3': 'GAS - April ',
                         'OIL.3': 'OIL - April',
                         'GAS.4': 'GAS - May',
                         'OIL.4': 'OIL - May',
                         'GAS.5': 'GAS - June',
                         'OIL.5': 'OIL - June',
                         'GAS.6': 'GAS - July',
                         'OIL.6': 'OIL - July',
                         'GAS.7': 'GAS - August',
                         'OIL.7': 'OIL - August',
                         'GAS.8': 'GAS - September',
                         'OIL.8': 'OIL - September',
                         'GAS.9': 'GAS - October',
                         'OIL.9': 'OIL - October',
                         'GAS.10': 'GAS - November',
                         'OIL.10': 'OIL - November',
                         'GAS.11': 'GAS - December',
                         'OIL.11': 'OIL - December',
                         }, inplace=True)
                         
    #some entity id's have multiple entries.  I do not know 
    #why this is.  it appears like they might be the production
    #attributed to different owners. try summing the production
    #for now...
    prod_data = prod_data.groupby(by=['ENTITY_ID','YEAR']).sum()
    prod_data.reset_index(inplace=True)
    
    
    #add a year column (may remove this when we try summing over the entire frame)
    #prod_data.loc[:,'Year'] = year
    annual_production_data[year] = prod_data
    
    
    #now clean the formation string.  These tend to have 
    #extraneous whitespace
    def clean_formation_string(in_string):
        
        out_string = str(in_string).strip()
        return " ".join(out_string.split())
        
    #data["FORMATION"] = data["FORMATION"].astype(str) 
    data["FORMATION"] = data["FORMATION"].apply(clean_formation_string)
    data["WELL_NAME"] = data["WELL_NAME"].apply(clean_formation_string)
    data["OPERATOR"] = data["OPERATOR"].apply(clean_formation_string)
    data["PURCHASER"] = data["PURCHASER"].apply(clean_formation_string)
    annual_data[year] = data

Cleaning 1988 data...


/Users/bhall/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Cleaning 1989 data...
Cleaning 1990 data...
Cleaning 1991 data...
Cleaning 1992 data...
Cleaning 1993 data...
Cleaning 1995 data...
Cleaning 1996 data...
Cleaning 1997 data...
Cleaning 1998 data...
Cleaning 1999 data...
Cleaning 2000 data...
Cleaning 2001 data...
Cleaning 2002 data...
Cleaning 2003 data...
Cleaning 2004 data...
Cleaning 2005 data...
Cleaning 2006 data...
Cleaning 2007 data...
Cleaning 2008 data...
Cleaning 2009 data...
Cleaning 2010 data...
Cleaning 2011 data...
Cleaning 2012 data...
Cleaning 2013 data...
Cleaning 2014 data...
Cleaning 2015 data...


In [31]:
annual_production_data[2005]

,ENTITY_ID,YEAR,GAS - January,OIL - January,GAS - February,OIL - February,GAS - March,OIL - March,GAS - April,OIL - April,...,GAS - August,OIL - August,GAS - September,OIL - September,GAS - October,OIL - October,GAS - November,OIL - November,GAS - December,OIL - December
0,35003000010000,2005,1950.0,NaN,1589.0,NaN,1842.0,NaN,1724.0,NaN,...,1702.0,NaN,1484.0,NaN,1626.0,NaN,1730.0,NaN,1342.0,NaN
1,35003000250000,2005,85.0,NaN,74.0,NaN,80.0,NaN,69.0,NaN,...,81.0,NaN,57.0,NaN,113.0,NaN,80.0,NaN,75.0,NaN
2,35003000680000,2005,488.0,NaN,408.0,NaN,312.0,NaN,635.0,NaN,...,933.0,NaN,1006.0,NaN,1214.0,NaN,1136.0,NaN,815.0,NaN
3,35003000710000,2005,1301.0,NaN,1057.0,NaN,1874.0,NaN,1826.0,NaN,...,2089.0,NaN,2333.0,NaN,2354.0,NaN,1885.0,NaN,1086.0,NaN
4,35003000730000,2005,573.0,NaN,440.0,NaN,509.0,NaN,476.0,NaN,...,4336.0,NaN,1859.0,NaN,488.0,NaN,180.0,NaN,260.0,NaN
5,35003000960000,2005,0.0,NaN,0.0,NaN,682.0,NaN,823.0,NaN,...,595.0,NaN,194.0,NaN,86.0,NaN,626.0,NaN,786.0,NaN
6,35003001020000,2005,340.0,NaN,806.0,NaN,879.0,NaN,795.0,NaN,...,986.0,NaN,926.0,NaN,312.0,NaN,1184.0,NaN,431.0,NaN
7,35003001430000,2005,776.0,NaN,701.0,NaN,761.0,NaN,739.0,NaN,...,760.0,NaN,887.0,NaN,883.0,NaN,713.0,NaN,771.0,NaN
8,35003200020000,2005,815.0,NaN,738.0,NaN,840.0,NaN,831.0,NaN,...,856.0,NaN,720.0,NaN,831.0,NaN,817.0,NaN,826.0,NaN
9,35003200140000,2005,2401.0,NaN,2043.0,NaN,2301.0,NaN,2258.0,NaN,...,2207.0,NaN,2089.0,NaN,2185.0,NaN,2059.0,NaN,2272.0,NaN


In [57]:
all_data = pd.concat(annual_data)
all_data

ALLOWABLE_CLASS  ALLOWABLE_TYPE API_COUNTY API_NUMBER        APR  \
1988 1      UO               I                     003      20183          4   
     2                                             003      20331          4   
     3                                             003      20404          4   
     4                                             003      20404          4   
     5      UG               I                     003      20607          4   
     6                                             003      21201          4   
     7      UO               I                     003      21256          4   
     8      AO               I                     003      21267          4   
     9      AO               I                     003      21273          4   
     10     AO               I                     003      21286          4   
     11     UO               I                     003      21401          4   
     12                                            003      21466          4   
     13                                            003      30122          4   
     14     UG               I                     007      00004          4   
     15     UG               S                     007      00025          4   
     16     UG               S                     007      00060          4   
     17     UG               I                     007      00062          4   
     18     UG               I                     007      00104          4   
     19     UG               I                     007      00106          4   
     20     UG               S                     007      00111          4   
     21                                            007      00114          4   
     22     UG               I                     007      00114          4   
     23     UG               R                     007      00167          4   
     24     UG               R                     007      00167          4   
     25     UG               I                     007      00173          4   
     26     UG               I                     007      00177          4   
     27     UG               I                     007      00179          4   
     28     UG               I                     007      00179          4   
     29     UG               I                     007      00186          4   
     30                                            007      00187          4   
...                     ...             ...        ...        ...        ...   
2015 41162              NaN             NaN        153      23527          4   
     41163              NaN             NaN        153      23528          4   
     41164              NaN             NaN        153      23532          4   
     41165              NaN             NaN        153      23544          4   
     41166              NaN             NaN        153      23547          4   
     41167              NaN             NaN        153      23552          4   
     41168              NaN             NaN        153      23567          4   
     41169              NaN             NaN        153      30010          4   
     41170              NaN             NaN        153      30013          4   
     41171              NaN             NaN        153      30019          4   
     41172              NaN             NaN        153      30020          4   
     41173              NaN             NaN        153      30039          4   
     41174              NaN             NaN        153      30066          4   
     41175              NaN             NaN        153      30086          4   
     41176              NaN             NaN        153      35159          4   
     41177              NaN             NaN        153      35174          4   
     41178              NaN             NaN        153      35182          4   
     41179              NaN             NaN        153      35199          4   
     41180  

## The Tidy Data

63884 wells in total.

In [58]:
temp = annual_data[1996]
temp[temp['ENTITY_ID'] == '35149201050000']
                                            

,API_COUNTY,API_NUMBER,SUFFIX,WELL_NAME,WELL_NO,OPER_NO,OPERATOR,ME,SECTION,TWP,...,OCT,OIL - October,GAS - October,NOV,OIL - November,GAS - November,DEC,OIL - December,GAS - December,ENTITY_ID
23974,149,20105,,HAGGARD,3-20,11187,BURLINGTON RES OIL & GAS LP,IM,20,10N,...,10,NaN,587.0,11,NaN,491.0,12,NaN,581.0,35149201050000
23975,149,20105,,HAGGARD,3-20,11187,BURLINGTON RES OIL & GAS LP,IM,20,10N,...,10,NaN,153475.0,11,NaN,141244.0,12,NaN,141627.0,35149201050000
23976,149,20105,,HAGGARD,3-20,11187,BURLINGTON RES OIL & GAS LP,IM,20,10N,...,10,NaN,1901.0,11,NaN,1987.0,12,NaN,1786.0,35149201050000
23977,149,20105,,HAGGARD,3-20,11187,BURLINGTON RES OIL & GAS LP,IM,20,10N,...,10,182.0,NaN,11,NaN,NaN,12,NaN,NaN,35149201050000


In [64]:
temp[temp['ENTITY_ID'] == '35149201050000']['PURCHASER']

23974      EL PASO NATURAL GAS CO
23975        ANR PIPELINE COMPANY
23976                 TRANSOK LLC
23977    MERIDIAN OIL TRADING INC
Name: PURCHASER, dtype: object

The are repeated entries for the same well that have different Purchasers.  What does this mean?